In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
from scipy.integrate import simpson
from scipy.stats import median_abs_deviation

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_100deg_nb13.npy')
pm_data_noerr = np.load('npy/pm_LAE_100deg_nb13.npy')
f_name = 'Source_cats/Source_cat_LAE_100deg_nb13.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/median_errors.npy')

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
nb_c = 13
N_nb = 6

pm_err = (np.ones(pm_data.shape).T * errors).T
cont_est, cont_err = stack_estimation(
    pm_data, pm_err, nb_c, N_nb, w_central
    )
real_cont_value = NB_synthetic_photometry(
    mock['SEDs_no_line'], mock['w_Arr'],
    w_central[nb_c], nb_fwhm(tcurves, nb_c, True)
)

In [ ]:
## Plots
np.random.seed(22)
for mock_n in np.random.randint(0, pm_data.shape[1], 20):
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots(figsize = (12, 8))
    ax.plot(mock['w_Arr'], mock['SEDs_no_line'][mock_n], c='orange', lw=2, zorder=-1)
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], c='gray', fmt='.')
    ax.scatter(w_central[nb_c], real_cont_value[mock_n], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4],
                xerr=bb_fwhm[-4]/2, yerr=errors[-4],
                fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], yerr=errors[-3],
                xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], yerr=errors[-2],
                xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], yerr=errors[-1],
                xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    # Fit line
    ax.errorbar(w_central[nb_c], cont_est[mock_n], yerr=cont_err[mock_n],
                c='violet', marker='^', markersize=9,
                capsize=4, label='Stack NBs', elinewidth=2, capthick=2)
    
    plt.legend()

    plt.show()
    print(mock['EW_Arr'][mock_n])

In [ ]:
for nb_c in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]:
    fig, ax = plt.subplots(2, figsize=(13, 10), sharex=True)
    for N_nb in np.arange(1, nb_c):
        pm_err = (np.ones(pm_data.shape).T * errors).T

        cont_est_stk, cont_err_stk = stack_estimation(
            pm_data, pm_err, nb_c, N_nb, w_central
        )

        real_cont_value = NB_synthetic_photometry(
        mock['SEDs_no_line'], mock['w_Arr'],
        w_central[nb_c], nb_fwhm(tcurves, nb_c, True)
    )
        
        mean_stk = np.average(
            ((cont_est_stk - real_cont_value) / real_cont_value)
            )
        abs_mean_stk = np.average(
            np.abs((cont_est_stk - real_cont_value) / real_cont_value)
            )

        ax[0].bar(N_nb, mean_stk, color='C1')
        ax[1].bar(N_nb, abs_mean_stk, color='C1')
        print('{0:0.5f}'.format(mean_stk), end=' ')
    plt.subplots_adjust(hspace=.0)
    ax[0].set_xticks(np.arange(1, nb_c))
    ax[0].set_ylim((-0.2, 1))
    ax[1].set_ylim((-0.2, 1))
    ax[0].set_title(filters_tags[nb_c], fontsize=20)
    plt.show()

In [ ]:
for N_nb_stack in range(1,13):
    fig, ax =  plt.subplots(figsize = (13, 10))

    N_points = 50
    sigma_min = 0.01
    sigma_max = 7

    sigma_Arr = np.linspace(sigma_min, sigma_max, N_points)
    sigma_hist_stk = np.zeros(N_points)

    ax.plot(erf(sigma_Arr / 2**0.5), erf(sigma_Arr / 2**0.5), linestyle='dashed', c='C1')

    N_sources = pm_data.shape[1]

    for nb_c in [10, 11, 12, 13, 14, 15, 16]:
        err = (np.ones(pm_data.shape).T * errors).T
        cont_stk, cont_err_stk = stack_estimation(
            pm_data, err, nb_c, N_nb_stack, w_central
            )
        real_cont_value = NB_synthetic_photometry(
            mock['SEDs_no_line'], mock['w_Arr'],
            w_central[nb_c], nb_fwhm(tcurves, nb_c, True)
        )
        ## Synthetic photometry of the continuum using the central NB
        w_t = tcurves['w'][nb_c]
        t = tcurves['t'][nb_c]
        w_Arr = mock['w_Arr']

        #### ERROR TESTING ####

        sigma_hist = np.zeros(N_points)

        for i in range(N_points):
            sigma_hist_stk[i] = len(
                np.where(
                    np.abs(cont_stk - real_cont_value) < sigma_Arr[i] * cont_err_stk
                )[0]
            )

        # Plot the error test
        ax.set_ylabel('N / N_sources', fontsize=15)
        ax.set_xlabel('$\sigma$', fontsize=15)

        ax.plot(
            erf(sigma_Arr / 2**0.5), sigma_hist_stk/N_sources, marker='.',
            linestyle='none', label=filters_tags[nb_c], markersize=12
        )

        print(
            '{}, integral = {}'.format(
                filters_tags[nb_c],
                simpson(
                    erf(sigma_Arr / 2**0.5),
                    sigma_hist_stk/N_sources - erf(sigma_Arr / 2**0.5)
                )
            )
        )

    ax.legend(fontsize=15)
    ax.set_xticks(np.linspace(0, 1, 11))
    ax.set_yticks(np.linspace(0, 1, 11))
    ax.set_title('N_nb = {}'.format(N_nb_stack), fontsize=20)
    ax.grid()
    plt.show()

In [ ]:
fig, ax =  plt.subplots(figsize = (13, 10))

N_points = 50
sigma_min = 0.01
sigma_max = 7

sigma_Arr = np.linspace(sigma_min, sigma_max, N_points)
sigma_hist_stk = np.zeros(N_points)

ax.plot(erf(sigma_Arr / 2**0.5), erf(sigma_Arr / 2**0.5), linestyle='dashed', c='C1')

N_sources = pm_data.shape[1]
nb_c = 14

for N_nb_stack in range(1, 13):
    err = (np.ones(pm_data.shape).T * errors).T
    cont_stk, cont_err_stk = stack_estimation(
        pm_data, err, nb_c, N_nb_stack, w_central
        )
    real_cont_value = NB_synthetic_photometry(
        mock['SEDs_no_line'], mock['w_Arr'],
        w_central[nb_c], nb_fwhm(tcurves, nb_c, True)
    )
    ## Synthetic photometry of the continuum using the central NB
    w_t = tcurves['w'][nb_c]
    t = tcurves['t'][nb_c]
    w_Arr = mock['w_Arr']

    #### ERROR TESTING ####

    sigma_hist = np.zeros(N_points)

    for i in range(N_points):
        sigma_hist_stk[i] = len(
            np.where(
                np.abs(cont_stk - real_cont_value) < sigma_Arr[i] * cont_err_stk
            )[0]
        )

    # Plot the error test
    ax.set_ylabel('N / N_sources', fontsize=15)
    ax.set_xlabel('$\sigma$', fontsize=15)

    ax.plot(
        erf(sigma_Arr / 2**0.5), sigma_hist_stk/N_sources, marker='.',
        linestyle='none', label='N_nb = {}'.format(N_nb_stack),
        markersize=13
    )    

ax.legend(fontsize=15)
ax.set_xticks(np.linspace(0, 1, 11))
ax.set_yticks(np.linspace(0, 1, 11))
ax.set_title(filters_tags[nb_c], fontsize=20)
ax.grid()
plt.show()

In [ ]:
## Plots
np.random.seed(25)
idx_arr = np.random.randint(0, pm_data.shape[1], 10)
nb_arr = np.arange(8, 21)
cont_est_arr = np.zeros((len(idx_arr), len(nb_arr))) # mock_n x nb_n
cont_err_arr = np.copy(cont_est_arr)
N_nb = 7

for i in range(len(nb_arr)):
    cont_est_arr[:, i], cont_err_arr[:, i] = stack_estimation(
            pm_data[:, idx_arr], pm_err[:, idx_arr], nb_arr[i], N_nb, w_central
        )

j = 0
for mock_n in idx_arr:
    pm = pm_data[:,mock_n]
    
    fig, ax = plt.subplots(figsize = (12, 8))
    ax.plot(mock['w_Arr'], mock['SEDs_no_line'][mock_n], c='orange', lw=2, zorder=-1)
    ax.errorbar(w_central[:-3], pm[:-3], yerr=errors[:-3], c='gray', fmt='.')
    ax.scatter(w_central[nb_c], real_cont_value[mock_n], c='black')

    ax.scatter(w_central[-4], pm[-4], c='purple' , marker='s')
    ax.scatter(w_central[-3], pm[-3], c='green'  , marker='s')
    ax.scatter(w_central[-2], pm[-2], c='red'    , marker='s')
    ax.scatter(w_central[-1], pm[-1], c='dimgray', marker='s')

    ax.errorbar(w_central[-4], pm[-4],
                xerr=bb_fwhm[-4]/2, yerr=errors[-4],
                fmt='none', color='purple' , elinewidth=4)
    ax.errorbar(w_central[-3], pm[-3], yerr=errors[-3],
                xerr=bb_fwhm[-3]/2, fmt='none', color='green'  , elinewidth=4)
    ax.errorbar(w_central[-2], pm[-2], yerr=errors[-2],
                xerr=bb_fwhm[-2]/2, fmt='none', color='red'    , elinewidth=4)
    ax.errorbar(w_central[-1], pm[-1], yerr=errors[-1],
                xerr=bb_fwhm[-1]/2, fmt='none', color='dimgray', elinewidth=4)

    ax.set_xlabel('$\lambda\ (\AA)$', size=15)
    ax.set_ylabel('$f_\lambda$ (erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$)', size=15)

    ## Continuum estimate
    for i in range(len(nb_arr)):
        ax.errorbar(
            w_central[nb_arr[i]], cont_est_arr[j, i],
            yerr=cont_err_arr[j, i],
            c='violet', marker='^', markersize=9,
            capsize=4, label='Stack NBs', elinewidth=2, capthick=2
        )
    j += 1
    
    plt.show()

In [ ]:
## EW distribution without selection
for nb_c in range(9, 18):
    fig, ax = plt.subplots(figsize=(12, 9))

    ax.hist(
        mock['EW_Arr'], bins=np.linspace(-50, 300, 15),
        density=False, label='Output EW$_0$'
        )

    for N_nb in range(1, 10):
        cont_est, cont_err = stack_estimation(
            pm_data, pm_err, nb_c, N_nb, w_central
        )
        EW_ret = (pm_data[nb_c] - cont_est) * nb_fwhm(tcurves, nb_c, True) / cont_est\
            / (w_central[nb_c] / 1215.67)

        hist, bins = np.histogram(EW_ret, bins=np.linspace(-50, 300, 15))

        ax.plot(
            [(bins[i] + bins[i+1]) / 2 for i in range(len(hist))],
            hist,
            marker='s', markersize=10, linestyle='--',
            label='N_nb = {}'.format(N_nb)
            )
    ax.legend(fontsize=15)
    ax.set_title(filters_tags[nb_c], fontsize=20)
    ax.set_xlabel('EW$_0$ ($\AA$)', fontsize=15)
    ax.set_ylabel('N', fontsize=15)
    plt.plot()

In [ ]:
## EW distribution with selection
for nb_c in range(9, 18):
    fig, ax = plt.subplots(figsize=(12, 9))

    ax.hist(
        mock['EW_Arr'], bins=np.linspace(10, 300, 25),
        density=False, label='Output EW$_0$'
        )

    for N_nb in range(1, 10):
        cont_est, cont_err = stack_estimation(
            pm_data, pm_err, nb_c, N_nb, w_central
        )

        EW_ret = (pm_data[nb_c] - cont_est) * nb_fwhm(tcurves, nb_c, True) / cont_est\
            / (w_central[nb_c] / 1215.67)
        line = (
            (pm_data[nb_c] - cont_est > 3 * (pm_err[nb_c]**2 + cont_err**2)**0.5)
            & (EW_ret > 10)
        )

        hist, bins = np.histogram(EW_ret[line], bins=np.linspace(10, 300, 25))

        ax.plot(
            [(bins[i] + bins[i+1]) / 2 for i in range(len(hist))],
            hist,
            marker='s', markersize=10, linestyle='--',
            label='N_nb = {}'.format(N_nb)
            )
    ax.legend(fontsize=15)
    ax.set_title(filters_tags[nb_c], fontsize=20)
    ax.set_xlabel('EW$_0$ ($\AA$)', fontsize=15)
    ax.set_ylabel('N', fontsize=15)
    plt.plot()

In [ ]:
## EW distribution with selection
fig, ax = plt.subplots(figsize=(12, 9))
ax.hist(
    mock['EW_Arr'], bins=np.linspace(0, 300, 15),
    density=False, label='Output EW$_0$'
    )
for N_nb in range(1, 11):
    hist = np.zeros(14)
    for nb_c in range(12, 15):
        cont_est, cont_err = stack_estimation(
            pm_data, pm_err, nb_c, N_nb, w_central
        )

        EW_ret = (pm_data[nb_c] - cont_est) * nb_fwhm(tcurves, nb_c, True) / cont_est\
            / (w_central[nb_c] / 1215.67)
        line = (
            (pm_data[nb_c] - cont_est > 3 * (pm_err[nb_c]**2 + cont_err**2)**0.5)
            & (EW_ret > 10)
        )

        hist_i, bins = np.histogram(EW_ret[line], bins=np.linspace(0, 300, 15))
        hist += hist_i

    ax.plot(
        [(bins[i] + bins[i+1]) / 2 for i in range(len(hist))],
        hist,
        marker='s', markersize=10, linestyle='--',
        label='N_nb = {}'.format(N_nb)
        )
ax.legend(fontsize=15)
ax.set_title(filters_tags[nb_c-1:nb_c+2], fontsize=20)
ax.set_xlabel('EW$_0$ ($\AA$)', fontsize=15)
ax.set_ylabel('N', fontsize=15)
plt.plot()

In [ ]:
## EW diff
w_lya = 1215.67

for N_nb in range(1, 11):
    fig, ax = plt.subplots(figsize=(12, 9))
    EW_ret_Arr = np.ones(mock['EW_Arr'].shape) * -999.
    for nb_c in [13]:
        fwhm = nb_fwhm(tcurves, nb_c, True)
        cont_est, cont_err = stack_estimation(
            pm_data, pm_err, nb_c, N_nb, w_central
        )

        EW_ret = (pm_data[nb_c] - cont_est) * fwhm / cont_est\
            / (w_central[nb_c] / w_lya)
        line = (
            (pm_data[nb_c] - cont_est > 3 * (pm_err[nb_c]**2 + cont_err**2)**0.5)
            & (EW_ret > 10)
        )
        EW_ret_Arr[line] = EW_ret[line]

    mask999 = np.where(EW_ret_Arr != -999.)

    ax.plot(
        mock['EW_Arr'][mask999], EW_ret_Arr[mask999],
        marker='.', markersize=12, linestyle='', alpha=0.7
        )
    ax.plot(
        np.linspace(0, 300, 300), np.linspace(0, 300, 300), linestyle='--'
        )
    ax.text(
        25, 250, 'bias = {0:0.2f} \nsigma = {1:0.2f}'.format(
            np.median(
                EW_ret_Arr[mask999] - mock['EW_Arr'][mask999],
                ),
            median_abs_deviation(
                EW_ret_Arr[mask999] - mock['EW_Arr'][mask999],
                ),
            ),
        fontsize=15
    )

    ax.set_ylim((0, 300))
    ax.set_xlim((0, 300))
    ax.set_xlabel('Real EW$_0$ ($\AA$)', fontsize=15)
    ax.set_ylabel('Retrieved EW$_0$ ($\AA$)', fontsize=15)
    ax.set_title('N_nb = {}'.format(N_nb), fontsize=20)
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

ax.plot(np.log10(pm_data_noerr[-3]), mock['EW_Arr'], marker='.', linestyle='')

plt.show()